## Imports

In [23]:
import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Data

In [24]:
fundamental_equity = pd.read_csv("../Results/fundamental_equity.csv", index_col=0, parse_dates=True)
macroRS_equity = pd.read_csv("../Results/macroRS_equity.csv", index_col=0, parse_dates=True)

In [25]:
fundamental_equity.rename(columns={"w_GB": "w_UK"}, inplace=True)
fundamental_equity

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_UK,w_JP,w_US
Date,,,,,,,,,,,,,
2000-10-31,-0.014490,-0.014490,-0.017086,0.516370,0.0,0.006621,0.000000,0.000000,0.000000,0.000000,-0.671423,0.154763,0.193538
2000-11-30,0.014938,0.014938,-0.060660,0.564591,0.0,0.004208,0.000000,0.000000,0.000000,0.000000,-0.140549,-0.281921,0.025523
2000-12-31,0.013422,0.013422,0.015919,0.411406,0.0,0.006017,0.000000,0.000000,0.000000,0.000000,0.313632,0.024381,-0.049763
2001-01-31,-0.003768,-0.003768,0.019408,0.091338,0.0,0.006319,0.000000,0.000000,0.000000,0.000000,0.333450,0.152697,-0.018989
2001-02-28,-0.000051,-0.000051,-0.084288,0.238369,0.0,0.003807,0.000000,0.000000,0.000000,0.000000,0.359363,0.003083,-0.264665
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.004282,0.004282,0.043488,0.434028,0.0,0.004008,0.257928,-0.229885,-0.554724,-0.352211,0.244677,-0.331347,0.969176
2025-07-31,0.002353,0.002353,0.013121,1.024281,0.0,0.004409,0.139950,0.086605,-0.382956,-0.214398,0.249965,0.097009,0.099018
2025-08-31,-0.008566,-0.008566,0.026408,1.295210,0.0,0.000000,-0.157670,0.623461,-0.042112,0.076925,-0.388934,-0.364045,0.198893


In [26]:
macroRS_equity

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_AU,w_CH,w_JP,w_UK,w_EM,w_EU
Date,,,,,,,,,,,,,
2002-09-30,-0.066517,-0.066517,-0.109650,0.000000,0.0,0.0014,0.238968,-0.420918,-0.245958,-0.085291,0.079001,0.000000,0.367682
2002-10-31,0.073873,0.073873,0.073988,0.199622,0.0,0.0014,0.273424,-0.440851,-0.087465,-0.235395,0.090860,0.000000,0.473299
2002-11-30,0.017593,0.017593,0.054137,0.558988,0.0,0.0012,0.081494,-0.390202,-0.238383,0.238288,-0.079381,0.000000,0.405777
2002-12-31,-0.018405,-0.018405,-0.048213,0.169611,0.0,0.0011,0.218959,-0.380326,-0.226122,0.075627,-0.075640,0.000000,0.369097
2003-01-31,-0.026167,-0.026167,-0.030204,0.322039,0.0,0.0010,0.225187,-0.396305,-0.074403,-0.223941,0.071217,0.000000,0.372078
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.010496,-0.010496,0.043488,0.280533,0.0,0.0034,0.270900,0.401230,-0.260789,-0.131643,0.131515,-0.421709,0.000000
2025-07-31,0.004953,0.004953,0.013121,0.423485,0.0,0.0034,0.261889,0.384474,0.124684,-0.126988,0.000000,-0.388985,-0.250121
2025-08-31,0.018012,0.018012,0.026408,0.795353,0.0,0.0038,-0.133035,0.410752,0.275015,0.140009,-0.270440,0.000000,-0.404289
